# 01_naive_rag Playground

이 노트북은 **Metadata Filtered RAG (V1)** 파이프라인을 실험하는 공간입니다.
Router Agent가 질문의 의도를 파악하여 카테고리를 추출하고, 이를 바탕으로 필터링된 검색을 수행합니다.

**사전 준비**:
1. `.env` 파일 설정 완료
2. `python 01_naive_rag/ingest.py` 실행하여 데이터 적재 완료

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
from dotenv import load_dotenv

load_dotenv()

# Add root directory to sys.path to import config
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from pipeline import RAGPipelineV1
from config import Config

## 1. Milvus 적재 상태 확인

In [2]:
from pymilvus import MilvusClient

client = MilvusClient(uri=Config.MILVUS_URI, token=Config.MILVUS_TOKEN)
collection_name = Config.MILVUS_COLLECTION_NAME_V1

if client.has_collection(collection_name):
    count = client.query(collection_name=collection_name, filter="", output_fields=["count(*)"])[0]["count(*)"]
    print(f"Collection '{collection_name}' exists with {count} entities.")
else:
    print(f"Collection '{collection_name}' does not exist. Please run ingest.py first.")

Collection 'audit_reports_v1' exists with 13064 entities.


## 2. 파이프라인 초기화 및 라우터 테스트

In [3]:
pipeline = RAGPipelineV1()
print("Pipeline Initialized!")

Pipeline Initialized!


### 2.1. 라우터 (Category Extraction) 테스트
질문에서 카테고리가 잘 추출되는지 확인합니다.

In [4]:
queries = [
    "예산 낭비 사례 알려줘",
    "인사 채용 비리 관련 내용은?",
    "계약 규정 위반 사례"
]

for q in queries:
    category = pipeline.route_query(q)
    print(f"Q: {q} -> Category: {category}")

Q: 예산 낭비 사례 알려줘 -> Category: 예산·회계·재정
Q: 인사 채용 비리 관련 내용은? -> Category: 인사·복무·조직
Q: 계약 규정 위반 사례 -> Category: 계약·구매·입찰


### 2.2. 전체 답변 생성 테스트

In [5]:
query = "한국철도공사의 예산 관련 지적사항은?"
answer = pipeline.run(query)

print("\n=== Answer ===\n")
print(answer)

Processing query (V1): 한국철도공사의 예산 관련 지적사항은?
Detected Category: 예산·회계·재정
Applying Filter: cat_L1 == '예산·회계·재정'

=== Answer ===

한국철도공사와 관련된 예산 관련 지적 사항은 다음과 같습니다.

1. **경영 평가 성과급 등 과다 지급 부적정**: 2019년 한국철도공사에 지급된 경영 평가 성과급 등이 과다 지급되었으며, 이는 예산 편성 지침에 따른 지급 기준을 위반한 것으로 나타났습니다.

2. **단순 노무 용역 퇴직 급여 충당금 정산 부적정**: 한국철도공사가 위탁한 단순 노무 용역의 퇴직 급여 충당금 정산이 부적정하게 이루어졌습니다. 이는 관련 법률에 따라 실제 충당 금액에 따라 정산 지급되어야 함에도 불구하고, 누락된 부분이 발견되었습니다.

위와 같은 지적 사항들은 예산 집행의 투명성과 효율성을 저해할 수 있으므로 주의가 필요합니다.
